In [1]:
import numpy as np
import pandas as pd
import pgeocode as pg
from utils import listdir_nohidden, antibiotics
import os

nomi = pg.Nominatim('gb')

path = 'raw_gp/'

month_folders = listdir_nohidden(path) # path to downloaded data

addresses = []
BNFs = []
ChemSubstances = []
gp_datas = []

i=0
for month in month_folders:
    address = pd.read_csv(path+month+'/Address.csv')
    bnf = pd.read_csv(path+month+'/BNF.csv')
    chem = pd.read_csv(path+month+'/ChemSubstance.csv')

    gp_path = [f for f in os.listdir(path+month) if f.startswith("GPData")][0]
    gp_data = pd.read_csv(path+month+'/'+gp_path, delimiter=',')

    antibiotic_mask = gp_data['BNFCode'].str.startswith(tuple(antibiotics.values()))
    gp_data = gp_data.loc[antibiotic_mask]

    zips = address['Postcode'].apply(lambda x: x[:4]) # get 1st 4 chars of postcode
    geo_query = nomi.query_postal_code(zips.to_numpy()) # lookup coords from postcodes

    # Add coords
    address['lat'] = geo_query['latitude']
    address['long'] = geo_query['longitude']
    
    addresses.append(address)
    BNFs.append(bnf)
    ChemSubstances.append(chem)
    gp_datas.append(gp_data)

    print(gp_path+' done!')

    #if i==10: break
    i+=1

addresses = pd.concat(addresses, axis=0)
gp_datas = pd.concat(gp_datas, axis=0)
BNFs = pd.concat(BNFs, axis=0)
ChemSubstances = pd.concat(ChemSubstances, axis=0)

GPData202006.csv done!
GPData202202.csv done!
GPData202205.csv done!
GPData202008.csv done!
GPData202204.csv done!
GPData202009.csv done!
GPData202203.csv done!
GPData202007.csv done!
GPData202110.csv done!
GPData201806.csv done!
GPData201906.csv done!
GPData202111.csv done!
GPData202102.csv done!
GPData202105.csv done!
GPData201809.csv done!
GPData202104.csv done!
GPData201909.csv done!
GPData202103.csv done!
GPData201903.csv done!
GPData202210.csv done!
GPData201902.csv done!
GPData201910.csv done!
GPData202012.csv done!
GPData202002.csv done!
GPData202211.csv done!
GPData201810.csv done!
GPData202005.csv done!
GPData202208.csv done!
GPData202206.csv done!
GPData202201.csv done!
GPData201911.csv done!
GPData201912.csv done!
GPData201807.csv done!
GPData201811.csv done!
GPData202207.csv done!
GPData202003.csv done!
GPData202004.csv done!
GPData202209.csv done!
GPData201907.csv done!
GPData201812.csv done!
GPData201904.csv done!
GPData201901.csv done!
GPData201804.csv done!
GPData20211

In [2]:
print(gp_datas.columns)

red_gp_data = gp_datas.drop(columns=['Locality',
                                     'ActCost',
                                     'DDD',
                                     'ADQ',
                                     'ActCost',
                                     'Items',
                                     'NIC'])

Index(['HB', 'Locality', 'PracticeID', 'BNFCode', 'BNFName', 'Items', 'NIC',
       'ActCost', 'Quantity', 'DDD', 'ADQ', 'Period'],
      dtype='object')


In [3]:
print(addresses.columns)

red_addresses = addresses.drop(columns=['Locality',
                                        'Street',
                                        'Area',
                                        'Posttown',
                                        'County',
                                        'PsuPracticeCode',
                                        '0Period',
                                        'Period'])

addresses_w_cluster_id = red_addresses.dropna(subset='ClusterID').drop_duplicates()

addresses_w_cluster_id['ClusterID'] = addresses_w_cluster_id['ClusterID'].astype(int)

addresses_w_cluster_id

Index(['Period', 'PracticeId', 'Locality', 'Street', 'Area', 'Posttown',
       'County', 'Postcode', 'lat', 'long', 'PsuPracticeCode', 'ClusterID',
       '0Period'],
      dtype='object')


,PracticeId,Postcode,lat,long,ClusterID
0,W93001,NP11 5GX,51.643900,-3.131700,60103
1,W93004,NP7 5DL,51.851292,-3.038624,60106
2,W93008,NP13 2AB,51.729800,-3.134300,60101
3,W94635,LL74 8TF,53.320400,-4.226100,60401
4,W93017,NP23 4EY,51.777100,-3.207900,60100
...,...,...,...,...,...
474,W00189,SA72 6SY,51.693333,-4.921600,60307
672,W00190,LL19 9LG,53.327400,-3.408033,60405
673,W00191,LL57 2PW,53.203925,-4.106675,60409
674,W00192,LL49 9AQ,52.942910,-4.160000,60410


In [4]:
merged = pd.merge(red_gp_data, addresses_w_cluster_id, 
                  left_on='PracticeID',
                  right_on='PracticeId',
                  how='left')

merged.head()

,HB,PracticeID,BNFCode,BNFName,Quantity,Period,PracticeId,Postcode,lat,long,ClusterID
0,7A6,W00118,0501011P0AAAJAJ,Phenoxymethylpenicillin Pot_Tab 250mg,56,202006,W00118,NP4 0TB,51.718516,-3.030521,60100.0
1,7A6,W00118,0501012G0AAABAB,Fluclox Sod_Cap 500mg,104,202006,W00118,NP4 0TB,51.718516,-3.030521,60100.0
2,7A6,W00118,0501013B0AAABAB,Amoxicillin_Cap 500mg,45,202006,W00118,NP4 0TB,51.718516,-3.030521,60100.0
3,7A6,W00118,0501030I0AAABAB,Doxycycline 100mg capsules,52,202006,W00118,NP4 0TB,51.718516,-3.030521,60100.0
4,7A6,W00118,0501030L0AAABAB,Lymecycline 408mg capsules,56,202006,W00118,NP4 0TB,51.718516,-3.030521,60100.0


In [5]:
print(merged.shape)

(739674, 11)


In [6]:
merged.to_csv('FilteredGPData.csv.zip', index=False, compression='gzip')